In [4]:
# Import necessary libraries
import jax.numpy as np
from jax import random, grad, jit
from jax.scipy.linalg import inv
from jax.numpy.linalg import norm
from tqdm import tqdm
from sklearn.datasets import make_spd_matrix
from jax_models import Lorenz96
from jax_models import visualize_observations, Lorenz96, generate_true_states, generate_gc_localization_matrix
from jax_filters import ensrf_steps
from jax_vi import var_cost, KL_gaussian, log_likelihood, KL_sum


# Parameters
F = 8.0
dt = 0.01
num_steps = 300  # Number of time steps
n = 40   # Number of state variables
Q = 0.1 * np.eye(n)  # Process noise covariance
R_matrix = make_spd_matrix(n)  # Generating a symmetric positive definite matrix for R
R = np.array(R_matrix)  # Observation noise covariance
inv_R = inv(R)
H = np.eye(n)  # Observation matrix
observation_interval = 1
initial_state = random.normal(random.PRNGKey(0), (n,))  # Initial state

l96_model = Lorenz96(dt = 0.01, F = 8)
# Generate true states and observations using the Lorenz '96 model
key = random.PRNGKey(0)
observations, true_states = generate_true_states(key, num_steps, n, initial_state, H, Q, R, l96_model.step, observation_interval)


# Define the gradient of the cost function
var_cost_grad = grad(var_cost, argnums=0)

# Initial guess for K and optimization parameters
K_opt = np.eye(n) * 0.4
alpha = 1e-6

# Optimization loop
for i in tqdm(range(50)):
    key, _ = random.split(key)
    grad_K = var_cost_grad(K_opt, m, C, Q, R, H, key) # needs to be redone
    K_opt -= alpha * grad_K

    # Optional: Evaluate performance after each update
    # e.g., recompute the estimated states and their cost

print("Optimized K:", K_opt)


AttributeError: 'numpy.ndarray' object has no attribute 'at'